In [1]:
# import qiskit,
# want to do operation on pauli strings
from qiskit.quantum_info import Pauli

z = Pauli("XX")
g = Pauli("ZZ")

# want to do operation on pauli strings
z.commutes(g)

# why is it true that z commutes with g?

True

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import StabilizerState, Pauli


qc = QuantumCircuit(3, 2)
# bell state prep
qc.h(1)
qc.cx(1, 2)
qc.barrier()

stab = StabilizerState(qc)
print(stab)

# apply a CNOT
qc.cx(0, 1)
stab = StabilizerState(qc)
print(stab)

# apply H to qubit 0
qc.h(0)
stab = StabilizerState(qc)
print(stab)


# measure qubit 1
qc.measure(1, 0)
# classically control-X on qubit 2
qc.x(2).c_if(qc.cregs[0], 1)

stab = StabilizerState(qc)
print(stab)

# # measure qubit 0
# qc.measure(0, 0)
# # classically control-Z on qubit 2
# qc.z(2).c_if(qc.cregs[0], 1)

# stab = StabilizerState(qc)
# print(stab)

In [ ]:
qc.draw()

In [7]:
# given a list of error operations, want to reverse-engineer the logical qubit code space
from qiskit.quantum_info import Pauli
import numpy as np
from qutip import *
from tqdm import tqdm

# # we have 4 qubits, thus 8 logical spaces
# # we have 1 logical, thus 7 error spaces

# # define errors
# error_list = [Pauli('ZII'), Pauli('IZI'), Pauli('IIZ')]

# # each error acting on the logical space must go to an orthogonal space

# # iterate over all possible logical spaces, check orthogonality condition

# # example, basis is |000> and |111>
# # create statevector
# from sympy import Symbol
# a = Symbol('a')
# b = Symbol('b')
# state = np.array([a, 0, 0, 0, 0, 0, 0, b]).reshape(8, 1)
# # complex conjugate transpose
# statet = state.conj().T

# # check <state|error|state> = 0
# ot = statet @ (Pauli('XII').to_matrix() @ state)
# print(ot == 0)

# # check <state|error|state> = 0
# ot = statet @ (Pauli('ZII').to_matrix() @ state)
# print(ot == 0)


error_list = [Pauli("XII"), Pauli("IXI"), Pauli("IIX")]
# want to iterate through all possible logical spaces, check if we can find one that works for phase flip
from sympy import symbols

a, b, c, d, e, f, g, h = symbols("a b c d e f g h")
state = np.array([a, 0, 0, 0, 0, 0, 0, h]).reshape(8, 1)
statet = state.conj().T

# check <state|error|state> = 0
ot = statet @ (Pauli("XII").to_matrix() @ state)
print(ot)
ot = statet @ (Pauli("IXI").to_matrix() @ state)
print(ot)
ot = statet @ (Pauli("IIX").to_matrix() @ state)
print(ot)

# should also check that all errors are orthogonal to each other
# check <state|error_1 error_2|state> = 0
# ot = statet @ (Pauli('XII').to_matrix().conj()) @ (Pauli('IXI').to_matrix() @ state)
# print(ot)
for i in range(len(error_list)):
    for j in range(i + 1, len(error_list)):
        # print error |state>
        # print("\n")
        # print(error_list[i].to_matrix() @ state)
        # print("\n")
        ot = (
            statet
            @ (error_list[i].to_matrix().conj())
            @ (error_list[j].to_matrix() @ state)
        )
        print(ot)

[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]


In [8]:
num_qubits = 3
a, b, c, d, e, f, g, h = symbols("a b c d e f g h")
state_list = []

for i in range(2 ** (2**num_qubits)):
    hot_encode = bin(i)[2:].zfill(2**num_qubits)
    # enforce weight at least 2
    if hot_encode.count("1") < 2:
        continue
    hot_encode = [int(x) for x in hot_encode]
    # multiply binary string to get state
    state = np.array([a, b, c, d, e, f, g, h]) * hot_encode
    state = state.reshape(2**num_qubits, 1)
    statet = state.conj().T
    state_list.append((state, statet))

In [9]:
error_list = [Pauli("III"), Pauli("XII"), Pauli("IXI"), Pauli("IIX")]
for i in range(len(state_list)):
    # want to check that <state| error_1 error_2 |state> = 0
    # unless error_1 = error_2, then <state| error_1 error_1 |state> = 1
    # if state satisfies this for all errors, print state
    for j in range(len(error_list)):
        for k in range(j + 1, len(error_list)):
            ot = (
                state_list[i][1]
                @ (error_list[j].to_matrix().conj())
                @ (error_list[k].to_matrix() @ state_list[i][0])
            )
            if ot != 0:
                break
        if ot != 0:
            break
    if ot == 0:
        print(state_list[i][0].T)

[[0 0 0 d e 0 0 0]]
[[0 0 c 0 0 f 0 0]]
[[0 b 0 0 0 0 g 0]]
[[a 0 0 0 0 0 0 h]]


In [10]:
num_qubits = 3
a, b, c, d, e, f, g, h = symbols("a b c d e f g h")
state_list = []

for i in range(2 ** (2**num_qubits)):
    hot_encode = bin(i)[2:].zfill(2**num_qubits)
    # enforce weight at least 2
    if hot_encode.count("1") < 2:
        continue
    hot_encode = [int(x) for x in hot_encode]
    # multiply binary string to get state
    state = np.array([a, b, c, d, e, f, g, h]) * hot_encode
    state = state.reshape(2**num_qubits, 1)
    statet = state.conj().T
    state_list.append((state, statet))

# # we know logical space a|+++> + b|---> is a solution
# # XXX not constructed by our state_list (?)
# state = [a+h, 0, 0, 0, 0, 0, 0, a-h]
# state = np.array(state).reshape(8, 1)
# statet = state.conj().T
# state_list.append((state, statet))

In [11]:
error_list = [Pauli("III"), Pauli("ZII"), Pauli("IZI"), Pauli("IIZ")]
for i in tqdm(range(len(state_list))):
    # want to check that <state| error_1 error_2 |state> = 0
    # unless error_1 = error_2, then <state| error_1 error_1 |state> = 1
    # if state satisfies this for all errors, print state
    for j in range(len(error_list)):
        for k in range(j + 1, len(error_list)):
            ot = (
                state_list[i][1]
                @ (error_list[j].to_matrix().conj())
                @ (error_list[k].to_matrix() @ state_list[i][0])
            )
            if ot != 0:
                break
        if ot != 0:
            break
    if ot == 0:
        print(state_list[i][0].T)

# problem, we don't find the expected solution |psi> = a|+++> + b|--->

100%|██████████| 248/248 [00:01<00:00, 167.81it/s]


In [12]:
num_qubits = 4
a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p = symbols(
    "a b c d e f g h i j k l m n o p"
)
state_list = []

for i in range(2 ** (2**num_qubits)):
    hot_encode = bin(i)[2:].zfill(2**num_qubits)
    # enforce weight at least 2
    if hot_encode.count("1") < 2:
        continue
    hot_encode = [int(x) for x in hot_encode]
    # multiply binary string to get state
    state = (
        np.array([a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p]) * hot_encode
    )
    state = state.reshape(2**num_qubits, 1)
    statet = state.conj().T
    state_list.append((state, statet))

In [13]:
error_list = [
    Pauli("IIII"),
    Pauli("IXII"),
    Pauli("IIXI"),
    Pauli("IIIX"),
    Pauli("ZIII"),
    Pauli("IZII"),
    Pauli("IIZI"),
    Pauli("IIIZ"),
]
for i in tqdm(range(len(state_list))):
    # want to check that <state| error_1 error_2 |state> = 0
    # unless error_1 = error_2, then <state| error_1 error_1 |state> = 1
    # if state satisfies this for all errors, print state
    for j in range(len(error_list)):
        for k in range(j + 1, len(error_list)):
            ot = (
                state_list[i][1]
                @ (error_list[j].to_matrix().conj())
                @ (error_list[k].to_matrix() @ state_list[i][0])
            )
            if ot != 0:
                break
        if ot != 0:
            break
    if ot == 0:
        print(state_list[i][0].T)

100%|██████████| 65519/65519 [27:21<00:00, 39.92it/s]
